<h1><center> Vigenér cipher decryption </center><h1>

---

In [ ]:
import os
import sys

__module_path = os.path.abspath(os.path.join('..'))
if __module_path not in sys.path:
    sys.path.append(__module_path)

import cryptoanalysis

import numpy as np

from IPython.display import Javascript, HTML

from plotly import tools, graph_objs, __version__

# offline plotly
import plotly.offline as plotly

# online plotly
# import plotly.plotly as plotly

from ipywidgets import Layout, widgets, Box, HBox, VBox, interactive_output

In [ ]:
%matplotlib inline
%autosave 0

In [ ]:
plotly_version = __version__

# For online connection:
# tools.set_credentials_file(username='CermakM', api_key='q2yoGYw052dyHMd8ztqx')
# tools.set_config_file(world_readable=False, sharing='public')

# For offline connection:
plotly.init_notebook_mode(connected=True)

In [ ]:
_submitted = False
_reset = True

cipher = ''
decrypted_text = ''

analyser = cryptoanalysis.decryption.Analyser(lang='en')

### History

<p align="justify">
The __Vigenère cipher__ (French pronunciation: _viʒnɛːʁ_) is a method of encrypting alphabetic text by using a series of interwoven Caesar ciphers based on the letters of a keyword. It is a form of polyalphabetic substitution.<sup>[1][source]</sup>
    
Though the cipher is easy to understand and implement, for three centuries it resisted all attempts to break it; this earned it the description le chiffre indéchiffrable (French for 'the indecipherable cipher'). Many people have tried to implement encryption schemes that are essentially Vigenère ciphers. Friedrich Kasiski was the first to publish a general method of deciphering a Vigenère cipher in 1863.

The Vigenère cipher was originally described by Giovan Battista Bellaso in his 1553 book La cifra del. Sig. Giovan Battista Bellaso; however, the scheme was later misattributed to Blaise de Vigenère in the 19th century, and thus acquired its present name.
</p>
[source]: https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher

### Encrypted text

In [ ]:
def _create_text_box(*args, **kwargs):
    for widget in args:
        widget.layout=Layout(width='100%', height='200px')
    box = Box(args, **kwargs)
    box.layout.display = 'flex'
    box.layout.align_items = 'stretch'
    box.layout.min_height = '230px'
    box.layout.flex_direction = 'row'
    return box

In [ ]:
input_area = widgets.Textarea(
    placeholder='Encrypted text',
)  

output_area = widgets.Textarea(
    placeholder='Decrypted text',
    disabled=True
)

input_box = _create_text_box(input_area)
output_box = _create_text_box(output_area)

In [ ]:
submit_input_button = widgets.Button(
    description='SUBMIT',
    disabled=False,
    button_style='success',
    icon='check'
)

reset_input_button = widgets.Button(
    description='RESET',
    disabled=False,
    button_style='danger'
)

In [ ]:
def _on_submit(sender):
    global analyser, cipher, _submitted, _reset
            
    cipher = input_area.value.lower()
    
    if not cipher:
        return
    
    _reset = False

    analyser = cryptoanalysis.decryption.Analyser(cipher=cipher, lang='en')
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _submitted:
#         Javascript('$.notify("Cipher submitted");')
        
        _submitted = True
    
def _on_reset(sender):
    global analyser, cipher, _submitted
    
    _submitted = False
    
    cipher = ''
    analyser = cryptoanalysis.decryption.Analyser(lang='en')
    input_area.value = ''
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _reset:
#         Javascript('$.notify("Analyser has been reset")')
        
        _reset = True

In [ ]:
submit_input_button.on_click(_on_submit)
reset_input_button.on_click(_on_reset)

In [ ]:
submit_input_button.layout.margin = '2px 2px 2px auto'
reset_input_button.layout.margin = '2px 2px 2px 5px'

submit_box = HBox([submit_input_button, reset_input_button])

submit_box.layout.display = 'flex'
submit_box.layout.flex_flow = 'row'
submit_box.layout.align_items = 'flex-end'

In [ ]:
button_samples = widgets.ToggleButtons(
    options=['Sample 1', 'Sample 2', 'Sample 3'],
    description='Load sample:',
    disabled=False,
    button_style='',
    tooltips=['Caesar cipher', 'Vigener cipher - short', 'Vigener cipher - long']
)

button_load = widgets.Button(
    description='LOAD',
    disabled=False,
    button_style='success',
    icon='upload'
)

samples = ['samples/TIKcipher1.txt', 'samples/TIKcipher2a.txt', 'samples/TIKcipher2b.txt']

def _on_sample_load(sender):
    sample_to_read = samples[button_samples.index]
    with open(os.path.join(__module_path, sample_to_read)) as s:
        input_area.value = s.read()
        
button_load.on_click(_on_sample_load)

In [ ]:
def _on_input(sender):
    pass
    
input_area.observe(_on_input, names='value')

In [ ]:
display(HBox([button_samples, button_load]))

In [ ]:
display(VBox([input_box, submit_box]))

---
### Frequency Analysis

In [ ]:
default_lang_trace = graph_objs.Bar(
    x=analyser.alphabet,
    y=analyser.letter_frequency,
    name='Language'
)

if cipher:
    cipher_dict = analyser.get_char_frequency()
    cipher_trace = graph_objs.Bar(
        x=list(cipher_dict.keys()),
        y=list(cipher_dict.values()),
        name='Cipher'
    )
    data = [default_lang_trace, cipher_trace]

else:
    data = [default_lang_trace]
    
layout = graph_objs.Layout(
    showlegend=True,
    title='Frequency analysis',
    xaxis=dict(tickangle=-45),
    yaxis=dict(
        title='Letter frequency',
        tickformat=' %'),
    barmode='group'
)


---

In [ ]:
fig = graph_objs.Figure(data=data, layout=layout)

plotly.iplot(fig, filename='cipher-plot', show_link=False)

---

### Key decryption

In [ ]:
%%HTML
<style>
.MathJax_Display {
    text-align: center !important;
}
</style>

<h4>1) Key length analysis using Kasiski's examination method</h4><br>
The Kasiski examination involves searching for strings of characters that are repeated in the ciphertext. The strings should be three characters long or more for the examination to be successful. Then, the distances between consecutive occurrences of the strings are likely to be multiples of the length of the keyword.~<sup>[1][source]</sup>
    
[source]: https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher
    
Our approach ivolves:
<br>
   - Shift matrix of dimensions $l \times l$ where $l$ is length of the cipher
     Each row consists of a cipher shifted by one
   - Lay shift matrix over its duplicate and count coincident letters
   - Let amount of coincident letters for each row be a coincidence value
   - Coincidence index is distance of the highest coincidence values from the original cipher string

<h4>2) Key analysis</h4><br>
The goal of this part is to find an ideal correlation of $i$th cipher character frequency with frequency of the alphabet (english in our case), where $i$ is a position of key character.
This process has to be repeated for each key character:
<br>
   - Vector of character frequency of dimension $n$, where $n$ is the size of the alphabet 
   - Shift matrix $\mathbf{S}$ of dimensions $n \times n$
     Each row consits of a cipher character frequency shifted by one
   - Perform matrix multiplication of $\mathbf{S}$ and $\mathbf{L}$, where $\mathbf{L}$ is a vector of language frequency
     Result is a vector of frequency scores for each shift $\mathbf{Y}$
     
    $$ \mathbf{S} \times \mathbf{L}^T = \mathbf{Y}^T $$
    
    \begin{equation*}
    \begin{bmatrix}
    s_1 &  s_2 & s_3 & \dots & s_n \\
    s_2 &  s_3 & s_4 & \dots & s_n \\
    s_3 &  s_4 & s_5 & \dots & s_n \\
    s_4 &  s_5 & s_6 & \dots & s_n \\
    \end{bmatrix}~ \times ~ \begin{bmatrix} 
    l_1 \\
    l_2 \\
    \vdots \\
    l_n \\
    \end{bmatrix}~ = ~ \begin{bmatrix}
    y_1 \\
    y_2 \\
    \vdots \\
    y_n \\
    \end{bmatrix}
    \end{equation*} 
    
<h4>3) Reviewing shift and considering rotation</h4><br>
Correctness of the max value of shift vector depends immensly on the length of the ciphered text provided for the analysis. Hence the maximum value might not be the correct one.
Also, rotation might have an effect on the resulting shift, it is necessary that the corresponding shift was reviewed before submitting.

---

In [ ]:
widget_rotation = widgets.ToggleButtons(
    options=[1, 0],
    disabled=False,
    tooltips=['Rotation 0: `a` -> `a`', 'Rotation 1: `a` -> `b`']
)
widget_rotation.index = 1

options = [] if not cipher else analyser.get_key_len_list()

widget_key_len = widgets.ToggleButtons(
    options=['-'] if not options else options,
    disabled=not options,
    button_style=''
)

key = ''

In [ ]:
def _on_rot_change(sender):
    global key
    if _submitted:
        key, _ = analyser.get_keys(key_id=widget_key_len.index, rot=widget_rotation.value)
        
        display(Javascript('IPython.notebook.execute_cells_below()'))
    else:
        key = ''

    display(Javascript('IPython.notebook.execute_cells_below()'))

In [ ]:
def _on_len_change(sender):
    global key
    if _submitted:
        key, _ = analyser.get_keys(key_id=widget_key_len.index, rot=widget_rotation.value)
        
        display(Javascript('IPython.notebook.execute_cells_below()'))
    else:
        key = ''

In [ ]:
if not _submitted:
    _on_len_change(None)  # Run for initialization

In [ ]:
widget_key_len.observe(_on_len_change, 'index')
widget_rotation.observe(_on_rot_change, 'index')

In [ ]:
display(widgets.Label(value="Select rotation:"))
display(widget_rotation)

In [ ]:
display(widgets.Label(value="Select key length:"))
display(widget_key_len)

In [ ]:
toggle_button_layout = Layout(
    display='block',
    margin='0 auto',
    justify_content='center',
    min_width='200px',
)

button_up = widgets.Button(
    disabled=False,
    icon='angle-up',
    layout=toggle_button_layout,
    button_style='success'
)

button_down = widgets.Button(
    disabled=False,
    icon='angle-down',
    layout=toggle_button_layout,
    button_style='success'
)

checkbox_use_suggested_keys = widgets.ToggleButton(
    value=True,
    disabled=False,
    icon='check',
    description='Suggested only',
    tooltip='Use only suggested key characters instead of whole alphabet'
)

In [ ]:
key_char_list = [c for c in key] or ['-']

In [ ]:
_custom_key = ''
_use_custom_key = False
_toggled_key_index = 0
_toggled_key_button = None

In [ ]:
key_char_list = [c for c in key] or ['-']

In [ ]:
button_keys = [widgets.ToggleButton(description=k, tooltip=str(i)) for i, k in enumerate(key_char_list)]
button_keys_dict = dict((i, k) for i, k in enumerate(button_keys))

box_layout = Layout(
    overflow_x='auto',
    justify_content='center',
    width='80%',
    margin='0 auto',
)

key_box = Box(button_keys, layout=box_layout)

button_use_custom_key = widgets.Button(
    display='block',
    description='USE KEY',
    tooltip='Use custom key',
    disabled=False,
    button_style='success',
    icon='key',
)

button_reset_key = widgets.Button(
    display='block',
    description='RESET KEY',
    tooltip='Reset custom key',
    disabled=False,
    button_style='info',
    icon='key',
)

In [ ]:
from itertools import cycle
from collections import deque

# Dictionary of possible key variants
key_char_vectors = dict()
for index, char in enumerate(key_char_list):
    if char == '-':
        break
    key_char_vectors[index] = cycle(analyser.get_shift_vector(index))
    next(key_char_vectors[index])  # skip one at init

In [ ]:
shift_layout = graph_objs.Layout(
    showlegend=True,
    title='Corresponding shift',
    xaxis=dict(
        tickangle=-45,
        domain=[0, 1]
    ),
    xaxis2=dict(
        tickangle=-45,
        anchor='y2',
        domain=[0, 1]
    ),
    yaxis=dict(
        title='Letter frequency',
        tickformat=' %',
        domain=[0.60, 1]
    ),
    yaxis2=dict(
        title='Letter frequency',
        tickformat=' %',
        anchor='x2',
        domain=[0, 0.40]
    ),
#     height='600px'
)

plot_control = widgets.FloatSlider(
    value=0,
    min=0.0,
    max=1e5,  # Should be sufficient for number of updates
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
)

shift_dict = analyser.default_alphabet_dct

alphabet_deque = deque(analyser.alphabet)
alpha_d = alphabet_deque.copy()

In [ ]:
from collections import Counter

def _plot_shift(*args, **kwargs):

    if not _submitted:
        return
    
    shift_letters = analyser.alphabet 
    shift_freq = [0.0 for _ in analyser.alphabet]
    
    if _toggled_key_button is not None:

        key_len = len(key_char_list)
        # Get letters with stride key_len
        cipher_strip = cipher.replace(' ', '')[_toggled_key_index::key_len]

        # bag
        cipher_strip_bag = Counter(cipher_strip)
        cipher_strip_bag.update(shift_dict)
        
        # sort
        sorted_items = sorted([(k, v) for k, v in cipher_strip_bag.items()])
        # shift
        shift_strip = deque(sorted_items)
        shift_strip.rotate(ord('a') - ord(_toggled_key_button.description))
        shift_letters = [t[0] for t in shift_strip]
        # freq
        shift_count = [t[1] for t in shift_strip]
        c_sum = sum(shift_count)
        shift_freq = [c / c_sum for c in shift_count]

    cipher_shift_trace = graph_objs.Bar(
        x=shift_letters,
        y=shift_freq,
        name='Shifted trace',
        xaxis='x2',
        yaxis='y2',
        marker=dict(
            color='orange',
        )
    )
    
    shift_fig = graph_objs.Figure(data=[default_lang_trace, cipher_shift_trace], layout=shift_layout)

    plotly.iplot(shift_fig, show_link=False)

In [ ]:
def _trigger_plot():
    # Trigger
    plot_control.value += 0.1
    

def _on_toggle_up(sender):
    global alpha_d
    if not key_char_vectors:
        return
    index, button = _toggled_key_index, _toggled_key_button
    if checkbox_use_suggested_keys.value:
        alpha_d = alphabet_deque.copy()
        shift = int(next(key_char_vectors[index])) + widget_rotation.value
    else:
        shift = -1

    deque.rotate(alpha_d, shift)
    
    button.description = alpha_d[0]

    _trigger_plot()
    
    
def _on_toggle_down(sender):
    global alpha_d
    if not key_char_vectors:
        return
    index, button = _toggled_key_index, _toggled_key_button
    if checkbox_use_suggested_keys.value:
        alpha_d = alphabet_deque.copy()
        for i in range(4):  # 4 is magic (len of shift vector, stable on 5)
            shift = int(next(key_char_vectors[index]))
        shift += widget_rotation.value
    else:
        shift = 1
        
    deque.rotate(alpha_d, shift)
    
    button.description = alpha_d[0]
    
    _trigger_plot()
    
    
def _on_toggle_key(sender):
    global _toggled_key_index, _toggled_key_button, alpha_d
    if _toggled_key_button is not None:
        # No need to re-draw plot nor trigger
        if _toggled_key_button.tooltip == sender.owner.tooltip:
            return
        
        # Reset old triggers
        _toggled_key_button.value = False
    
    # Set new triggers
    try:
        _toggled_key_index, _toggled_key_button = int(sender.owner.tooltip), sender.owner
    except IndexError:
        pass
    
    if not checkbox_use_suggested_keys.value:
        alpha_d = alphabet_deque.copy()
        alpha_d.rotate(-(ord(sender.owner.description) - ord('a')))
    
    _trigger_plot()
    
        
def _on_custom_key(sender):
    global _use_custom_key, _custom_key
    _use_custom_key = True
    _custom_key = "".join([b.description for b in key_box.children])
    output_area.value = analyser.decipher(custom_key=_custom_key, rot=widget_rotation.value)
    
def _on_reset_key(sender):
    global key, _custom_key, _use_custom_key
    _custom_key = key
    _use_custom_key = False
    for i, but in enumerate(key_box.children):
        but.description = key[i]
    output_area.value = analyser.decipher(custom_key=key, rot=widget_rotation.value)
    
    _trigger_plot()

In [ ]:
button_up.on_click(_on_toggle_up)
button_down.on_click(_on_toggle_down)

for button in key_box.children:
    button.observe(_on_toggle_key, names='value')

button_use_custom_key.on_click(_on_custom_key)
button_reset_key.on_click(_on_reset_key)

#### Decrypted key

In [ ]:
button_custom_box = Box([button_use_custom_key, button_reset_key])
button_custom_box.layout.display = 'flex'
button_custom_box.layout.flex_flow = 'row'
button_custom_box.layout.justify_content = 'flex-end'

In [ ]:
display(widgets.Label(value="Decrypted key:"))
display(checkbox_use_suggested_keys)

In [ ]:
display(button_up)
display(key_box)
display(button_down)
display(button_custom_box)

### Corresponding shift

In [ ]:
interactive_output(_plot_shift, controls=dict(control_button = plot_control))

In [ ]:
# Set first index to True to draw graph
key_box.children[0].value = True

### Text decryption

Deciphering monoalphabetic cipher is just reversed encoding process.<br>
Key is applied to each character of the cipher, respectively, and reversed shift is performed.

---

In [ ]:
result = ''
if _submitted:
    use_key = _custom_key if _use_custom_key else key
    # Rotation has been handled by rotating the key
    result = analyser.decipher(custom_key=use_key, rot=widget_rotation.value)
    
output_area.value = result

In [ ]:
display(widgets.Label(value="Decrypted text:"))
display(_create_text_box(output_area))